In this project, we aim to investigate the sources of spatial heterogeneity in glioblastoma multiforme using mathematical modeling.
We include the following sources of heterogeneity:
- genetic heterogeneity
- phenotypic heterogeneity (go-or-grow)

To this end, we adapt a CA model of glioblastoma multiforme from [1] that includes the go-or-grow hypothesis.
We then investigate the effects of genetic heterogeneity on the growth dynamics of the tumor.

### Plan
Things to consider:
- Should we include oxygen? If so, include a hypoxic region in the center of the tumor?
- Should we include directed motion, e.g., based on an oxygen gradient?
Let's start with this:
1. Change go-or-grow model to include a realistic mutation probability
2. A mutation might change: Proliferation rate, or phenotypic switch parameter
 

In [1]:
from numpy import dtype

from lgca import get_lgca
import numpy as np

In [2]:
%matplotlib qt

In [53]:
capacity = 50
std_kappa = 2.
r_d = 0.1
r_b = 0.5
kappa = 4
theta = .2
r_mut = 0.001
L = 150

In [54]:
nodes0 = np.zeros((L, L, 7))
nodes0[L//2-1:L//2+2, L//2-1:L//2+2, -1] = capacity

In [55]:
lgca = get_lgca(geometry='hex', dims=(L, L), interaction='go_or_grow_glioblastoma', ve=False, ib=True, capacity=capacity, kappa_std=std_kappa, r_d=r_d, r_b=r_b, kappa=kappa, theta=theta, nodes=nodes0, r_m=r_mut)

fitness increase for driver mutations set to  1.1


In [56]:
lgca.timeevo(300, record=True, recordfampop=True)

  0%|          | 0/300 [00:00<?, ?it/s]

In [39]:
lgca.animate_density()

In [57]:
cutoff_rel = 0.01
_, _, _, fcmap = lgca.muller_plot(cutoff_rel=cutoff_rel)

In [58]:
cells_t = lgca.nodes_t.sum(-1)

In [59]:
families_loc_t = np.zeros(cells_t.shape, dtype=int)
for ind, cells in np.ndenumerate(cells_t):
    if len(cells) == 0:
        continue
    else:
        values, counts = np.unique([lgca.props['family'][cell] for cell in cells], return_counts=True)
        ind2 = np.argmax(counts)
        families_loc_t[ind] = values[ind2]

In [60]:
t = -1
import matplotlib.pyplot as plt
plt.figure()
# print(np.unique(families_loc_t[t]))
_, pc, _ = lgca.plot_scalarfield(families_loc_t[t], cmap='tab20', cbar=True)
pc.set_facecolor([fcmap(famid) for famid in families_loc_t[t].ravel()])
pc.set_alpha(np.heaviside(lgca.dens_t[t], 0))
pc.set_edgecolor('none')

In [61]:
lgca.propagate_family_prop_to_cells('kappa')
lgca.propagate_family_prop_to_cells('r_b')
lgca.calc_prop_mean_spatiotemp()

{'family': masked_array(
   data=[[[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --]],
 
         ...,
 
         [[--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          [--, --, --, ..., --, --, --],
          ...,
          [--, --, --, ..., -

In [62]:
plt.figure()
lgca.plot_prop_spatial(propname='kappa')

(<Figure size 800x800 with 2 Axes>,
 <matplotlib.cm.ScalarMappable at 0x195df2129c0>)

In [63]:
plt.figure()
lgca.plot_density()

(<Figure size 800x606 with 2 Axes>,
 <matplotlib.cm.ScalarMappable at 0x195b587f800>)